In [ ]:
# Clone the PyVision repository for ready-to-use code
# Author: Pranjal Datta

!git clone https://github.com/pranjaldatta/PyVision.git
!git clone https://github.com/pranjaldatta/vehicle-detection-assignment.git

# Fixed tqdm for colab
# IGNORE the errors
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip -qqq

In [14]:

import cv2
import numpy as np 
from google.colab.patches import cv2_imshow

import os
from IPython.display import HTML
from base64 import b64encode
from tqdm.autonotebook import tqdm
import torch

In [5]:
# Instantiate the model with pretrained with weights
# We use the YOLOv3 model for object detection

from PyVision.pyvision.detection import yolov3


# checking if CUDA is enabled
if torch.cuda.is_available():
  device = "gpu"
else:
  device = "cpu"

yolo = yolov3.YOLOv3(device=device, confidence=0.8, nms_thresh=0.6)

Weight file exists.


In [7]:
path = "D:\Projects\Personal\vehicle-detection\road-traffic.mp4"


In [30]:
saveName = "detect.mp4"
compressedName = "detect_compressed.mp4"

In [10]:
size = 800
fourcc = cv2.VideoWriter_fourcc(*'MP4V')

writer = cv2.VideoWriter(filename=saveName, fourcc=fourcc, fps=30, frameSize=(size, size))

In [11]:
cap = cv2.VideoCapture(path)

In [ ]:
counter = 0
MAX_FRAMES = 700 
det_frames = []

if MAX_FRAMES > 3000:
  print("[WARN]: test video contains 3000 frames. Setting MAX_FRAMES to 3000")
  MAX_FRAMES = 3000

progress_bar = tqdm(range(0, MAX_FRAMES))
for idx,  _ in enumerate(progress_bar):
  
  ret, frame = cap.read()
  frame = cv2.resize(frame, (800, 800))
  
  try:
    time, imgs, objs = yolo.detect(frame, save=False, show=False)
  except:
    pass
  
  #print(f"Frame Id: {counter}/{MAX_FRAMES}: {time} secs")
  progress_bar.set_description(
      "Frame Id: {}/{}: {:.4f} secs".format(idx+1, MAX_FRAMES, time)
  )
  det_frames.append(imgs)
  



In [ ]:
## Write the detected frames into a video. The detected video is ready! Can be downloaded from the File Explorer tab!

for frame in tqdm(det_frames):
  writer.write(frame)
writer.release()

In [ ]:
if os.path.exists(compressedName):
  os.remove(compressedName)
compressed_path =  compressedName
os.system(f"ffmpeg -i {saveName} -vcodec libx264 {compressed_path}")

In [ ]:
if MAX_FRAMES > 700:
  print("Too many frames! Colab takes too much time to display large videos! Download the video from 'File Explorer' and play it locally!")
  raise ValueError("Too many frames to display! Please download and play the video locally!")

# Show video
mp4 = open(compressed_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=800 controls><source src={decoded_vid} type="video/mp4"></video>')